In [1]:
# first I want to make the geometry

import meep as mp
import math
import matplotlib
#matplotlib.use('agg')
import numpy as np
import matplotlib.pyplot as plt
from meep.materials import BK7
import h5py

# wvl_min = 0.760  # min wavelength
# wvl_max = 0.940  # max wavelength
# fmin = 1 / wvl_max  # min frequency
# fmax = 1 / wvl_min  # max frequency
# fcen = 0.5 * (fmin + fmax)  # center frequency
# df = fmax - fmin  # frequency width
# nfreq = 200  # number of frequency bins
wvl_cent = 0.830  # wavelength
fcen=1/wvl_cent
df =1 #pulse width (in frequency)

s = 20
resolution = 50
dpml = 2

cell_size = mp.Vector3(s,s,0)

boundary_layers = [mp.PML(thickness=dpml)]

beam_x0 = mp.Vector3(0,2.5)    # beam focus (relative to source center)
rot_angle = 0  # CCW rotation angle about z axis (0: +y axis)
beam_kdir = mp.Vector3(0,1,0).rotate(mp.Vector3(0,0,1),math.radians(rot_angle))  # beam propagation direction
beam_w0 = 0.3  # beam waist radius
beam_E0 = mp.Vector3(0,0,1)

sources = [mp.GaussianBeamSource(src=mp.GaussianSource(fcen,fwidth=df),
                                 center=mp.Vector3(0,-0.5*s+dpml+1.0),
                                 size=mp.Vector3(s),
                                 beam_x0=beam_x0,
                                 beam_kdir=beam_kdir,
                                 beam_w0=beam_w0,
                                 beam_E0=beam_E0,
                                 amplitude=3)]

In [2]:
# cover slip
thickness_slide=4
center_slide=-0.5*s+dpml+0.5*thickness_slide
thickness_medium=s-2*dpml-thickness_slide
center_medium=0.5*s-dpml-0.5*thickness_medium

distance_cylender_slide=0.5
Radius_cylinder=2.5
center_cylinder=-0.5*s+dpml+thickness_slide+distance_cylender_slide+Radius_cylinder



geometry = [mp.Block(mp.Vector3(s-2*dpml,thickness_slide,mp.inf),
                     center=mp.Vector3(0,center_slide,0),
                     material=BK7),
            mp.Block(mp.Vector3(s-2*dpml,thickness_medium,mp.inf),
                     center=mp.Vector3(0,center_medium,0),
                     material=mp.Medium(epsilon=1.79)),
            mp.Cylinder(radius=Radius_cylinder,center=mp.Vector3(0,center_cylinder,0),
                        axis=mp.Vector3(0,0,1),
                        material=mp.Medium(epsilon=1.96),
                        height=4),
            ]



In [3]:
sim = mp.Simulation(cell_size=cell_size,
                    boundary_layers=boundary_layers,
                    geometry=geometry,
                    sources=sources,
                    resolution=resolution,
                    force_complex_fields=True)

#
sim.run(mp.at_beginning(mp.output_epsilon),
        mp.to_appended("ez", mp.at_every(1, mp.output_efield_z)),
        mp.to_appended("poy",mp.at_every(1,mp.output_poynting)),
        mp.to_appended("energy",mp.at_every(1,mp.output_dpwr)),
        until=55)

# sim.run(until=2)
sim.use_output_directory()

-----------
Initializing structure...
time for choose_chunkdivision = 0.000720978 s
Working in 2D dimensions.
Computational cell is 20 x 20 x 0 with resolution 50
     block, center = (0,-6,0)
          size (16,4,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center = (0,2,0)
          size (16,12,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1.79,1.79,1.79)
     cylinder, center = (0,-1,0)
          radius 2.5, height 4, axis (0, 0, 1)
          dielectric constant epsilon diagonal = (1.96,1.96,1.96)
time for set_epsilon = 1.87764 s
lorentzian susceptibility: frequency=0.0982658, gamma=0
lorentzian susceptibility: frequency=7.0679, gamma=0
lorentzian susceptibility: frequency=12.9092, gamma=0
-----------
Meep: using complex fields.
creating output file "./eps-000000.00.h5"...
creating output file "./ez.h5"...
creating output file "./poy.h5"...
creating output file

'-out'

In [6]:
eps_data = sim.get_array(center=mp.Vector3(), size=cell_size, component=mp.Dielectric)

In [ ]:
hf=h5py.File('ez.h5','r')


n1 = hf.get('ez.i')
n1 = np.array(n1)
k=n1.shape
s=np.zeros((k[0],k[1]))

for i in range(k[2]):
    s = n1[:, :, i]
    plt.figure(i)
    plt.imshow(eps_data.transpose(), interpolation='spline36', cmap='binary')
    plt.imshow(s.transpose(), interpolation='spline36', cmap='RdBu', alpha=0.9)
    plt.axis('off')
    plt.show()